In [1]:
from gurobi import *
import pandas as pd
import numpy as np
import time

## 所有資訊前處理

In [2]:
begin = time.time()
url = r'/Users/yuchiaching/Desktop/OR109-2_student/data/OR109-2_case02_data_s1.xlsx'

In [3]:
# D_it
Demand_info = pd.read_excel(url, 'Demand').iloc[:,1:]
Demand = []
for i in Demand_info.index:
    Demand.append(list(Demand_info.loc[i]))

# I_it
Init_inv_info = pd.read_excel(url, 'Initial inventory')
In_transit_info = pd.read_excel(url, 'In-transit').iloc[:,2:]
In_transit = []
for i in In_transit_info.index:
    In_transit.append([list(Init_inv_info['Initial inventory'])[i]] + list(In_transit_info.loc[i]) + [0] * (26 - 3))

# C^V_ij
Shipping_cost_info = pd.read_excel(url, 'Shipping cost').iloc[:,1:]
print(Shipping_cost_info)
Shipping_cost = []
for i in Shipping_cost_info.index:
    Shipping_cost.append(list(Shipping_cost_info.loc[i]) + [0])

Price_and_cost_info = pd.read_excel(url, 'Price and cost').iloc[:,1:]
# C^P_i
Purchasing_cost = list(Price_and_cost_info['Purchasing cost'])
# C^H_i
Holding_cost = list(Price_and_cost_info['Holding cost'])

# C^F_j
Fixed_cost_shipping_method = [100, 80, 50]

# T_j
Lead_time = [1, 2, 3]

# V_i
Product_size_info = pd.read_excel(url, 'Size')
Product_size = list(Product_size_info['Size'])

Shortage_info = pd.read_excel(url, 'Shortage')
# C^L_i
Lost_sales_cost = list(Shortage_info['Lost sales'])
# C^B_i
Backorder_cost = list(Shortage_info['Backorder'])
# beta_i
Backorder_percentage = list(Shortage_info['Backorder percentage'])

Vendor_product_info = pd.read_excel(url, 'Vendor-Product')
Vendor_product = list(Vendor_product_info['Vendor'])

Vendor_cost_info = pd.read_excel(url, 'Vendor cost')
Vendor_cost = list(Vendor_cost_info['Ordering cost'])
# print(Vendor_cost)

Bounds_info = pd.read_excel(url, 'Bounds')
Bounds = list(Bounds_info['Minimum order quantity (if an order is placed)'])

Conflict_info = pd.read_excel(url, 'Conflict')

product1_list = list(Conflict_info['Product 1'] - 1)
product2_list = list(Conflict_info['Product 2'] - 1)

Conflict = []
for i in range(len(product1_list)):
    Conflict.append(tuple([(int(product1_list[i])), (int(product2_list[i]))]))

    Express delivery  Air freight
0                 44           18
1                 89           45
2                 86           38
3                 91           46
4                 50           21
..               ...          ...
95                71           71
96                77           59
97                80           48
98               115          107
99                24           22

[100 rows x 2 columns]


In [4]:
S_I = len(Product_size)  # Set of products
S_J = 3                  # Set of shipping methods
S_T = 26                 # Set of months
S_V = len(Vendor_cost)   # Set of vendors

# print(S_I, S_J, S_T, S_V)

## 預設訂購量格式

In [5]:
order = []
for i in range(S_I):
    order.append([])
    for j in range(S_J):
        order[i].append([])
        for t in range(S_T):
            order[i][j].append(0)
# print(order)

## 決定運送方式

In [6]:
# variable shipping cost
ocean_shipping_cost = []
for i in range(S_I):
    unit_cost = Product_size[i] / 0.5 * 1500
    ocean_shipping_cost.append(unit_cost)

product_demand = []
product_demand_from4 = []
for i in range(S_I):    
    product_demand.append(sum(Demand[i]))
    product_demand_from4.append(product_demand[i] - max(sum(In_transit[i]), sum(Demand[i][0:3])))

ocean_cost_sum_from4 = 0
for i in range(len(ocean_shipping_cost)):
    ocean_cost_sum_from4 += product_demand_from4[i]*ocean_shipping_cost[i]
ocean_cost_sum_from4 += 50 * len(Demand[0])

express_cost_sum_from4 = 0
for i in range(len(Shipping_cost)):
    express_cost_sum_from4 += product_demand_from4[i]*Shipping_cost[i][0]
express_cost_sum_from4 += 100*len(Demand[0])

air_cost_sum_from4 = 0
for i in range(len(Shipping_cost)):
    air_cost_sum_from4 += product_demand_from4[i]*Shipping_cost[i][1]
air_cost_sum_from4 += 80*len(Demand[0])

if express_cost_sum_from4 <= air_cost_sum_from4 and express_cost_sum_from4 <= ocean_cost_sum_from4:
    shipping_method = 0
elif air_cost_sum_from4 <= express_cost_sum_from4 and air_cost_sum_from4 <= ocean_cost_sum_from4:
    shipping_method = 1
else:
    shipping_method = 2

# 決定運送方式
print(shipping_method)

1


## 衝突產品字典

In [7]:
conflict_dict = dict()

for i in range(len(Conflict)):
    for j in range(len(Conflict[i])):
        if Conflict[i][j] not in conflict_dict:
            conflict_dict[Conflict[i][j]] = [Conflict[i][j-1]]
        else:
            conflict_dict[Conflict[i][j]].append(Conflict[i][j-1])


# 要檢查是否有衝突的產品清單
# 檢查順序按照有關聯的商品數排序
# 例如 conflict_dict[31] ＝ [78, 23] 因為跟 31 產品有關聯的產品有兩個所以要先檢查
check_conflict = list(conflict_dict.keys())
check_conflict = sorted(check_conflict, reverse=True, key=lambda d: len(conflict_dict[d]))

# # 範例
# print(conflict_dict[31])
# # 有衝突關係的產品
# print(check_conflict)

# 排序後結果
for i in check_conflict:
    print(i, conflict_dict[i])

36 [53, 88]
74 [83, 69]
31 [78, 23]
15 [42]
42 [15]
53 [36]
88 [36]
84 [92]
92 [84]
83 [74]
35 [52]
52 [35]
69 [74]
78 [31]
23 [31]


## 計算每期短缺量並比較
每期訂的量要能滿足 S_L+1 期的需求（包括當期）
S_L 為該訂購方式之 Lead time

In [8]:
# 該訂購方式的 leadtime
S_L = Lead_time[shipping_method]
print(S_L)

# 最後 S_L 期理論上不會訂貨
for t in range(S_T - S_L):

    inv_level = []
    lostcost = []

    # 計算若當期不訂購，則未來 S_L+1 期內的存貨水準
    for i in range(S_I):
        inv_level.append(sum(In_transit[i][t:(t+S_L+1)]) - sum(Demand[i][t:(t+S_L+1)]))

        # 如果有缺貨
        if inv_level[i] < 0:
            # 計算當期訂購量
            order[i][shipping_method][t] = int(-inv_level[i])
            # 計算當期缺貨損失
            lostcost.append(-(inv_level[i] * Backorder_percentage[i] * Backorder_cost[i] + inv_level[i] * (1-Backorder_percentage[i]) * Lost_sales_cost[i]))
        else:
            # 計算當期訂購量（沒有缺貨設為 0）
            order[i][shipping_method][t] = 0 
            # 計算當期缺貨損失（沒有缺貨設為 0）
            lostcost.append(0)

#     print(inv_level)

    # 檢查衝突問題   
    for i in check_conflict:
        # 要比較的對象
        temp_compare_index = [i] + conflict_dict[i]
        # 要比較的對象的訂購量
        temp_compare_order = []
        for k in temp_compare_index:
             temp_compare_order.append(order[k][shipping_method][t])

    #     # 檢查
    #     print(temp_compare_index)
    #     print(temp_compare_order)
    #     print('\n')

        # 如果組內有兩個產品以上要訂購，必須做處理
        if temp_compare_order.count(0) < (len(temp_compare_index) - 1):
            temp_compare_lostcost = []
            for k in temp_compare_index:
                temp_compare_lostcost.append(lostcost[k])

            # 要訂購的產品
            trgt = temp_compare_index[temp_compare_lostcost.index(max(temp_compare_lostcost))]

#             # 印出來檢查
#             print('有問題的產品組合')
#             print(temp_compare_index)
#             print('目前個別的訂購量')
#             print(temp_compare_order)
#             print('目前個別的缺貨成本')
#             print(temp_compare_lostcost)
#             print('要訂購的產品')
#             print(trgt)
#             print('\n')

            # 把其他不要訂購的產品的訂購量設為 0
            for k in temp_compare_index:
                if k != trgt:
                    order[k][shipping_method][t] = 0
    
    # 可能衝突商品處理完成
   
    # 處理其他部分
    for i in range(S_I):
        # 處理最低訂購量
        if (order[i][shipping_method][t] > 0) and (order[i][shipping_method][t] < Bounds[i]):
            order[i][shipping_method][t] = Bounds[i]
        
        # 更新下期實際期初存貨（也可以說是在途存貨）
        # 當期就有缺貨的話則更新 backorder 量到下期需求
        if (In_transit[i][t] - Demand[i][t]) > 0:
            In_transit[i][t+1] += (In_transit[i][t] - Demand[i][t])
        
        else:
            Demand[i][t+1] += int((Demand[i][t] - In_transit[i][t]) * Backorder_percentage[i])

        # 把訂購量更新到 S_L 期後的在途存貨
        In_transit[i][t+S_L] += order[i][shipping_method][t]


2


In [9]:
# real_shortage_list = []

# for i in range(S_I):
#     real_shortage_list.append([])
#     for t in range(S_T):
#         real_shortage_list[i].append(int(In_transit[i][t] - Demand[i][t]))

# real_shortage_pandas = pd.DataFrame(real_shortage_list)
# print(real_shortage_pandas)
# real_shortage_pandas.to_excel(r'/Users/yuchiaching/Desktop/real_shortage.xlsx')

## 花費時間

In [10]:
print(str(time.time() - begin) + ' seconds')

0.9336841106414795 seconds


## 匯出空運的訂購量成 Excel 檔案（方便查看和比較）

In [11]:
order_air = []

for i in range(S_I):
    order_air.append(order[i][1])

order_pandas = pd.DataFrame(order_air)
print(order_pandas)
order_pandas.to_excel(r'/Users/yuchiaching/Desktop/order_air.xlsx')

     0    1   2    3    4    5    6    7    8    9   ...   16   17   18   19  \
0     0    0   0   38   79   12  151  165  142  163  ...   49    0   67  148   
1     0    0   0   60  241  145   60  148   60  110  ...  180  156   60  239   
2     0    0  82  200   66  199  101    0  270  237  ...    0  196   61    0   
3     0    0  97  155  153   75  108  139   97  182  ...   70    0  158  229   
4     0    0  67   60   60   71  162   60  199    0  ...   60    0  134   60   
..  ...  ...  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
95    0    0  70   70   95  156  213  163  125  138  ...  197  251  202  108   
96    0    0   0    0    0  190   55    0  100  264  ...  136  144   52  101   
97    0    0   0    0    0    0    0    0    0    0  ...    0  292   30    0   
98    0    0  52  143   62  129   67  137   10  192  ...   28   38   63  115   
99  352  194   0  132   52  282  116  159   99  196  ...   19  322  120  277   

     20   21   22   23  24  25  
0   11